In [1]:
pip install pennylane==0.29.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from pennylane import numpy as np
import math
import pennylane as qml
import random
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import itertools

2024-03-15 16:03:14.328026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
tf.random.set_seed(15)
random.seed(42)
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams.update({'font.size':20})
legend_prop = {'weight':'bold'}
from pylab import rcParams
rcParams['axes.linewidth'] = 2

# Data

In [6]:
df = pd.read_csv('Processed_data_till_7_march.csv')
df = df.drop([ 'Unnamed: 0'], axis=1)
# Used data from Aug 2013 till 1st March 2024 
df=df[50:]
# Remove few redundant data
df=df.drop(columns=['Date'])
df=df.drop(columns=['CUMLOGRET_1'])
df=df.drop(columns=['Gold in USD volume'])
df=df.drop(columns=['Open'])
df=df.drop(columns=['High'])
df=df.drop(columns=['Low'])

In [7]:
close_min=min(df['Close'].values)
close_max=max(df['Close'].values)
a=-1
b=1
normalized_df_2=(b-a)*((df-df.min())/(df.max()-df.min()))+a

In [8]:
a=0
b=np.pi
normalized_df=(b-a)*((df-df.min())/(df.max()-df.min()))+a

In [9]:
normalized_df['Close']=normalized_df_2['Close']

In [10]:
df=normalized_df.copy()

In [11]:
df['add1']=0

In [12]:
# Set target and features
target = "Close"
features = list(df.columns.difference(["Close"]))
print(features)
print(len(features))

['3M', 'BBB_20_2.0', 'BBL_20_2.0', 'BBM_20_2.0', 'BBP_20_2.0', 'BBU_20_2.0', 'Close_copy', 'Crude Futures_close', 'Crude Futures_volume', 'Crude_H-L', 'Crude_O-C', 'EMA_14', 'EMA_21', 'EMA_7', 'FTSE_H-L', 'FTSE_O-C', 'GBP USD ', 'GBP_USD_H-L', 'GBP_USD_O-C', 'Gold in USD close', 'Gold_H-L', 'Gold_O-C', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'SMA_14', 'SMA_21', 'SMA_7', 'Volume', 'add1']
30


In [13]:
df.head()

,Close,Volume,EMA_7,EMA_14,EMA_21,SMA_7,SMA_14,SMA_21,MACD_12_26_9,MACDh_12_26_9,...,FTSE_H-L,FTSE_O-C,GBP_USD_H-L,GBP_USD_O-C,Gold_H-L,Gold_O-C,Crude_H-L,Crude_O-C,Close_copy,add1
50,-0.701430,0.662644,0.304164,0.215734,0.152416,0.368959,0.266307,0.182846,2.696624,1.665995,...,0.608132,1.346644,0.037320,1.899331,0.310536,1.453813,0.340350,1.005263,0.468992,0
51,-0.634817,0.592118,0.317588,0.223312,0.160106,0.378745,0.266077,0.191066,2.698435,1.711599,...,0.466823,1.012070,0.064779,1.956533,0.313124,1.125187,0.222025,1.002933,0.573629,0
52,-0.652298,0.568643,0.320097,0.225648,0.164150,0.374498,0.265523,0.194111,2.688490,1.722741,...,0.350243,1.377740,0.066751,2.569949,0.000000,1.334150,0.195436,1.145045,0.546169,0
53,-0.653357,0.697431,0.321521,0.227416,0.167648,0.372934,0.262472,0.198783,2.678158,1.730823,...,0.536468,1.306334,0.039261,1.956533,0.564141,1.268068,0.000000,0.994779,0.544505,0
54,-0.658919,0.780971,0.320184,0.227603,0.169891,0.372646,0.245023,0.204491,2.665253,1.731741,...,0.485496,1.325913,0.025333,2.116473,0.512385,1.378800,0.256592,1.016912,0.535768,0


In [14]:
sequence_length = 30
window =6

def create_dataset(dataset,target,features, lookback, window):
    X_store, y_store = [], []
    for i in range(0,len(dataset)-lookback-window,3):
        X = (dataset[features].values)[i:i+lookback]
        y = (dataset[target].values)[i+lookback:i+lookback+window]
        X_store.append(X)
        y_store.append(y)
    return np.array(X_store), np.array(y_store)

In [15]:
# Split train test data
from sklearn.model_selection import train_test_split
# df_train, df_test = train_test_split(df, test_size=0.05)
size=int(len(df))-10
df_train=df[:size]
df_test=df[size:]
print('Total sample', len(df))
print('Train sample', len(df_train))
print('Test sample', len(df_test))

Total sample 3212
Train sample 3202
Test sample 10


In [16]:
# Train data
X_train, y_train = create_dataset(
    df_train,
    target=target,
    features=features,
    lookback=sequence_length, window=window)

indices = tf.range(start=0, limit=tf.shape(X_train)[0])
shuffled_indices = tf.random.shuffle(indices)
X_train = tf.gather(X_train, shuffled_indices)
y_train = tf.gather(y_train, shuffled_indices)

print(X_train.shape, y_train.shape)

(1056, 30, 30) (1056, 6)


2024-03-15 16:03:16.441378: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


# QNN

In [17]:
nqubits = 10
device = qml.device('default.qubit', wires=nqubits)
# Define Quantum layer(1)
@qml.qnode(device)
def quantum_circuit_1(inputs, params):
    
    m=3
    k1=0 ; k2=k1+m ; k3=k2+m ; k4=k3+m ; k5=k4+m ; k6=k5+m ; k7=k6+m ; k8=k7+m ; k9=k8+m ; k10=k9+m ; k11=k10+m
        
    for p1,p2,p3,p4,p5,p6,p7,p8,p9,p10 in zip(params[k1:k2],params[k2:k3],params[k3:k4],
                                                  params[k4:k5],params[k5:k6],params[k6:k7],params[k7:k8],
                                                  params[k8:k9],params[k9:k10],params[k10:k11]):
        
        qml.Rot(*inputs[:3], wires=0)
        qml.Rot(*inputs[3:6], wires=1)
        qml.Rot(*inputs[6:9], wires=2)
        qml.Rot(*inputs[9:12], wires=3)
        qml.Rot(*inputs[12:15], wires=4)
        qml.Rot(*inputs[15:18], wires=5)
        qml.Rot(*inputs[18:21], wires=6)
        qml.Rot(*inputs[21:24], wires=7)
        qml.Rot(*inputs[24:27], wires=8)
        qml.Rot(*inputs[27:30], wires=9)
        
        qml.Rot(*p1, wires=0)
        qml.Rot(*p2, wires=1)
        qml.Rot(*p3, wires=2)
        qml.Rot(*p4, wires=3)
        qml.Rot(*p5, wires=4)
        qml.Rot(*p6, wires=5)
        qml.Rot(*p7, wires=6)
        qml.Rot(*p8, wires=7)
        qml.Rot(*p9, wires=8)
        qml.Rot(*p10, wires=9)
        
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4]) 
        qml.CNOT(wires=[4, 5])
        qml.CNOT(wires=[5, 6]) 
        qml.CNOT(wires=[6, 7])
        qml.CNOT(wires=[7, 8])
        qml.CNOT(wires=[8, 9])
        
    return [qml.expval(qml.PauliZ(k)) for k in range(nqubits)]

In [18]:
device = qml.device('default.qubit', wires=12)
# Define QNode
@qml.qnode(device)
def quantum_circuit_2(inputs, params):
    
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.RX(inputs[2], wires=2)
    qml.RX(inputs[3], wires=3)
    qml.RX(inputs[4], wires=4)
    qml.RX(inputs[5], wires=5)
    qml.RX(inputs[6], wires=6)
    qml.RX(inputs[7], wires=7)
    qml.RX(inputs[8], wires=8)
    qml.RX(inputs[9], wires=9)
    qml.RX(inputs[10], wires=10)
    qml.RX(inputs[11], wires=11)
    
    m=4
    k1=1234 ; k2=k1+m ; k3=k2+m ; k4=k3+m ; k5=k4+m ; k6=k5+m ; k7=k6+m ; k8=k7+m ; k9=k8+m ; k10=k9+m ; k11=k10+m
    k12=k11+m ; k13=k12+m 
    
#     print('Parameters in Second Qauntum circuit is till',k13)
#     print('Total parameter in Circuit 2',k13-k1)
        
    for p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12 in zip(params[k1:k2],params[k2:k3],params[k3:k4],
                                                  params[k4:k5],params[k5:k6],params[k6:k7],params[k7:k8],
                                                  params[k8:k9],params[k9:k10],params[k10:k11],params[k11:k12]
                                                      ,params[k12:k13]):
        
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 4]) 
        qml.CNOT(wires=[4, 5])
        qml.CNOT(wires=[5, 6]) 
        qml.CNOT(wires=[6, 7])
        qml.CNOT(wires=[7, 8])
        qml.CNOT(wires=[8, 9])
        qml.CNOT(wires=[9, 10])
        qml.CNOT(wires=[10, 11])
        
        qml.Rot(*p1, wires=0)
        qml.Rot(*p2, wires=1)
        qml.Rot(*p3, wires=2)
        qml.Rot(*p4, wires=3)
        qml.Rot(*p5, wires=4)
        qml.Rot(*p6, wires=5)
        qml.Rot(*p7, wires=6)
        qml.Rot(*p8, wires=7)
        qml.Rot(*p9, wires=8)
        qml.Rot(*p10, wires=9)
        qml.Rot(*p10, wires=10)
        qml.Rot(*p11, wires=11)
        
    return [qml.expval(qml.PauliZ(k)) for k in range(12)]

In [36]:
def loss_func(params,X,y):
    
    cost=0
    
    for j in range(len(X)):
        
        v1=[quantum_circuit_1(X[j][i].numpy(), params) for i in range(X.shape[1])]    

        # Flatten 
        flat_list = list(itertools.chain(*v1))
        v1_=np.array(flat_list)
    #     print('v1 shape is',v1_.shape)    
    #     print('Total parameter in Circuit 1',60)

        m1=300
        l1=30 ; l2=l1+m1 ; l3=l2+m1  ; l4=l3+m1 ; l5=l4+m1 
        l6=l5+1 ; l7=l6+1 ; l8=l7+1 
    #     print('Parameters in NN layer 1 start at',l1)

        v2= np.add(np.concatenate((params[l5],params[l6],params[l7],params[l8])),np.dot(np.transpose(
                                                                    v1_),
                                                                                       np.hstack((params[l1:l2],
                                                                                                  params[l2:l3],params[l3:l4],params[l4:l5])) ))


        v3=np.where(v2 > 0, v2, v2 * 0.05) 
    #     print('Parameters in NN layer 1 end at',l8)
    #     print('Total parameter in NN 1',l8-l1+1)

        l9=l8+1
    #     print('Parameters in Next circuit start at',l9)
        v4=quantum_circuit_2(np.array(v3),params)

        l11=1282
        m1=12
        l12=l11 ; l13=l12+m1 ; l14=l13+m1 ; l15=l14+m1 ; l16=l15+m1 ; l17=l16+m1
        l18=l17+1 ; l19=l18+1 ; l20=l19+1 ; l21=l20+1 
    #     print('Parameters in last layers start at',l12)
        v5= np.add(np.concatenate((params[l17],params[l18],params[l19],params[l20],params[l21])),np.dot(np.transpose(
                                                                    v4),
                                                                                       np.hstack((params[l12:l13],
                                                                                                  params[l13:l14],params[l14:l15],params[l15:l16],params[l16:l17])) ))
        v6=np.where(v5 > 0, v5, v5 * 0.05)

        m1=15
        l23=l21+1 ; l24=l23+m1 ; l25=l24+m1 
        l27=l25 ; l28=l27+1 
        v7 = np.add(np.concatenate((params[l27],params[l28])),np.dot(np.transpose(
                                                                    v6),
                                                                                       np.hstack((params[l23:l24],
                                                                                       params[l24:l25])) ))
    #     print('Last parameters is',l28)
    #     print('Total number of parameters in last is',l28-l12+1)

        cost=cost+np.mean((y[j] - v7) ** 2)

    return cost/len(X)

In [20]:
def cost_fn(params,**kwargs):
    z=kwargs['batch']
    
#     print('Batch is',z)
    
    v1=[quantum_circuit_1(X_train[z][i].numpy(), params) for i in range(X_train.shape[1])]    
    
    # Flatten 
    flat_list = list(itertools.chain(*v1))
    v1_=np.array(flat_list)
#     print('v1 shape is',v1_.shape)    
#     print('Total parameter in Circuit 1',60)
    
    m1=300
    l1=30 ; l2=l1+m1 ; l3=l2+m1  ; l4=l3+m1 ; l5=l4+m1 
    l6=l5+1 ; l7=l6+1 ; l8=l7+1 
#     print('Parameters in NN layer 1 start at',l1)
    
    v2= np.add(np.concatenate((params[l5],params[l6],params[l7],params[l8])),np.dot(np.transpose(
                                                                v1_),
                                                                                   np.hstack((params[l1:l2],
                                                                                              params[l2:l3],params[l3:l4],params[l4:l5])) ))

    
    v3=np.where(v2 > 0, v2, v2 * 0.05) 
#     print('Parameters in NN layer 1 end at',l8)
#     print('Total parameter in NN 1',l8-l1+1)
    
    l9=l8+1
#     print('Parameters in Next circuit start at',l9)
    v4=quantum_circuit_2(np.array(v3),params)
    
    l11=1282
    m1=12
    l12=l11 ; l13=l12+m1 ; l14=l13+m1 ; l15=l14+m1 ; l16=l15+m1 ; l17=l16+m1
    l18=l17+1 ; l19=l18+1 ; l20=l19+1 ; l21=l20+1 
#     print('Parameters in last layers start at',l12)
    v5= np.add(np.concatenate((params[l17],params[l18],params[l19],params[l20],params[l21])),np.dot(np.transpose(
                                                                v4),
                                                                                   np.hstack((params[l12:l13],
                                                                                              params[l13:l14],params[l14:l15],params[l15:l16],params[l16:l17])) ))
    v6=np.where(v5 > 0, v5, v5 * 0.05)
    
    m1=15
    l23=l21+1 ; l24=l23+m1 ; l25=l24+m1 
    l27=l25 ; l28=l27+1 
    v7 = np.add(np.concatenate((params[l27],params[l28])),np.dot(np.transpose(
                                                                v6),
                                                                                   np.hstack((params[l23:l24],
                                                                                   params[l24:l25])) ))
#     print('Last parameters is',l28)
#     print('Total number of parameters in last is',l28-l12+1)

    cost=np.mean((y_train[z] - v7) ** 2)
    return cost

In [21]:
params_1=np.random.uniform(0,np.pi, size=(30, 3),requires_grad = True)
params_2=np.random.normal(-0.05,0.05, size=(1204, 3),requires_grad = True)
params_3=np.random.normal(0,np.pi, size=(48, 3),requires_grad = True)
params_4=np.random.normal(-0.05,0.05, size=(97, 3),requires_grad = True)

params=np.vstack([params_1,params_2,params_3,params_4])

In [22]:
opt = qml.AdamOptimizer(stepsize=0.005)

In [23]:
len(params)

1379

In [24]:
# batch_size=32 # Batch size
batch_list=list(range(0,X_train.shape[0]))

In [25]:
batch_=list(range(len(batch_list)))

In [26]:

store_val=[]
store_params=[]

In [27]:
import time

In [ ]:
for i in range (5):
    print('Epoch',i,'running')
    batch_=list(range(len(batch_list)))
    while len(batch_)>0:
        j= random.choice(batch_)
        # start=time.time()
        params, prev_cost = opt.step_and_cost(cost_fn,params,batch=j)
        # end=time.time()
        # print(start-end)
        batch_.remove(j)
    store_val.append(i)
    store_params.append(params)
    print('-------------------------------------------')

Epoch 0 running
-------------------------------------------
Epoch 1 running
-------------------------------------------
Epoch 2 running


In [151]:
opt.stepsize

0.005

In [49]:
# Make pred for check data
def pred_(X,params):
    v1=[quantum_circuit_1(X[i], params) for i in range(X.shape[1])]    
    
    # Flatten 
    flat_list = list(itertools.chain(*v1))
    v1_=np.array(flat_list)
#     print('v1 shape is',v1_.shape)    
#     print('Total parameter in Circuit 1',60)
    
    m1=300
    l1=30 ; l2=l1+m1 ; l3=l2+m1  ; l4=l3+m1 ; l5=l4+m1 
    l6=l5+1 ; l7=l6+1 ; l8=l7+1 
#     print('Parameters in NN layer 1 start at',l1)
    
    v2= np.add(np.concatenate((params[l5],params[l6],params[l7],params[l8])),np.dot(np.transpose(
                                                                v1_),
                                                                                   np.hstack((params[l1:l2],
                                                                                              params[l2:l3],params[l3:l4],params[l4:l5])) ))

    
    v3=np.where(v2 > 0, v2, v2 * 0.05) 
#     print('Parameters in NN layer 1 end at',l8)
#     print('Total parameter in NN 1',l8-l1+1)
    
    l9=l8+1
#     print('Parameters in Next circuit start at',l9)
    v4=quantum_circuit_2(np.array(v3),params)
    
    l11=1282
    m1=12
    l12=l11 ; l13=l12+m1 ; l14=l13+m1 ; l15=l14+m1 ; l16=l15+m1 ; l17=l16+m1
    l18=l17+1 ; l19=l18+1 ; l20=l19+1 ; l21=l20+1 
#     print('Parameters in last layers start at',l12)
    v5= np.add(np.concatenate((params[l17],params[l18],params[l19],params[l20],params[l21])),np.dot(np.transpose(
                                                                v4),
                                                                                   np.hstack((params[l12:l13],
                                                                                              params[l13:l14],params[l14:l15],params[l15:l16],params[l16:l17])) ))
    v6=np.where(v5 > 0, v5, v5 * 0.05)
    
    m1=15
    l23=l21+1 ; l24=l23+m1 ; l25=l24+m1 
    l27=l25 ; l28=l27+1 
    v7 = np.add(np.concatenate((params[l27],params[l28])),np.dot(np.transpose(
                                                                v6),
                                                                                   np.hstack((params[l23:l24],
                                                                                   params[l24:l25])) ))
    return v7

In [38]:
def mae_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += abs(predicted[i] - actual[i])
    return sum_error/len(actual)

def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        sum_error += (predicted[i] - actual[i])**2
    return (sum_error/len(actual))**0.5

In [140]:
store_train_loss=[]

In [ ]:
for i in store_params:
    store_train_loss.append(loss_func(i,X_train,y_train))
    print(store_train_loss)

[0.014296596973371201]
[0.014296596973371201, 0.010890972032169967]
[0.014296596973371201, 0.010890972032169967, 0.012866807272178562]
[0.014296596973371201, 0.010890972032169967, 0.012866807272178562, 0.013608985374093182]


In [148]:
store_train_loss

[0.014296596973371201,
 0.010890972032169967,
 0.012866807272178562,
 0.013608985374093182,
 0.0140381687865124,
 0.00828122886020336,
 0.01660546928857549,
 0.007774903841803071,
 0.008424538907633805,
 0.008212622536320663,
 0.007699476254248118,
 0.007947194659143557,
 0.007787412414495354,
 0.00809867896191379,
 0.006418542528801139,
 0.005155190959455616,
 0.006798697711814243,
 0.011251551572487096,
 0.005868983060078206,
 0.00533633123773935]

In [149]:
store_params

[tensor([[ 2.30117846,  1.90200881,  2.16095629],
         [ 2.5601229 ,  0.12682885,  0.64811646],
         [ 0.35951692,  0.33589513,  1.97630631],
         ...,
         [-0.39883682, -0.34690981, -0.3722236 ],
         [ 0.1779886 ,  0.14512569,  0.1838868 ],
         [ 0.20251113,  0.19246362,  0.20966704]], requires_grad=True),
 tensor([[ 2.38815541,  1.91053491,  2.21454596],
         [ 2.5865741 ,  0.07090199,  0.6682875 ],
         [ 0.44300585,  0.27602166,  1.97630631],
         ...,
         [-0.28261206, -0.22102253, -0.21152323],
         [ 0.14945   ,  0.14583862,  0.16081868],
         [ 0.16889412,  0.16836558,  0.17650243]], requires_grad=True),
 tensor([[ 2.35104176,  1.87951724,  2.21223942],
         [ 2.59403785,  0.0227235 ,  0.67719112],
         [ 0.50925046,  0.18545821,  1.97630631],
         ...,
         [-0.23998414, -0.16405494, -0.14752233],
         [ 0.16732613,  0.16382788,  0.17573264],
         [ 0.18179346,  0.17333326,  0.19028023]], requires_grad

In [150]:
print(store_params[-1])

[[ 2.50539667  1.789174    2.3478578 ]
 [ 2.67460984 -0.04884651  0.39109681]
 [ 0.77655875  0.28014386  1.97630631]
 ...
 [ 0.01220465 -0.00560611  0.0144645 ]
 [ 0.1609896   0.16346478  0.1761056 ]
 [ 0.1732639   0.17417497  0.18066779]]


# From 8th to 15th march

In [66]:
df.tail()

,Close,Volume,EMA_7,EMA_14,EMA_21,SMA_7,SMA_14,SMA_21,MACD_12_26_9,MACDh_12_26_9,...,FTSE_H-L,FTSE_O-C,GBP_USD_H-L,GBP_USD_O-C,Gold_H-L,Gold_O-C,Crude_H-L,Crude_O-C,Close_copy,add1
3257,0.780294,1.014255,2.783765,2.809452,2.816693,2.788357,2.792277,2.781228,2.593007,1.866774,...,0.247744,1.150622,0.018482,2.022391,1.167099,0.530445,0.337691,0.810734,2.796479,0
3258,0.778182,0.813024,2.788291,2.813176,2.820330,2.787530,2.806508,2.784920,2.594404,1.871329,...,0.216606,1.310912,0.016793,1.994166,1.011831,0.703688,0.198095,1.086802,2.793161,0
3259,0.756231,1.223532,2.782193,2.811092,2.819937,2.777932,2.813133,2.786847,2.582949,1.847272,...,0.391072,1.397175,0.020487,1.881268,0.571904,0.980520,0.187459,1.103110,2.758681,0
3260,0.778182,1.072853,2.787112,2.814597,2.823279,2.777136,2.820101,2.786749,2.584843,1.856800,...,0.230737,1.206278,0.023846,1.956533,0.641775,0.869787,0.288500,0.892273,2.793161,0
3261,0.786889,1.123079,2.794566,2.819743,2.827785,2.778643,2.818461,2.790437,2.590150,1.871777,...,0.253447,1.263864,0.025189,1.994166,0.414048,1.116257,0.186129,0.980801,2.806839,0


In [134]:
X_8_15=(df[features].values)[len(df)-sequence_length:len(df)]

In [135]:
y_8_15=pred_(X_8_15,store_params[-1])

In [136]:
y_8_15

tensor([0.79303773, 0.79195144, 0.79769266, 0.80042047, 0.78445317,
        0.78092519], requires_grad=True)

In [137]:
b=1 ; a=-1

In [138]:
y_8_15=((((y_8_15-a)/(b-a))*(close_max - close_min))+close_min)

In [139]:
y_8_15

tensor([7701.7455762 , 7700.10507082, 7708.77544876, 7712.89499155,
        7688.78117726, 7683.45322574], requires_grad=True)